In [1]:
!pip install -U transformers datasets torch accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86

In [2]:
from datasets import load_dataset
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments
import random

In [3]:
# Cargar un subconjunto del conjunto de datos
dataset = load_dataset("cnn_dailymail", "3.0.0", split='train[:10%]')  # Usar solo el 10% de los datos para entrenar
validation_dataset = load_dataset("cnn_dailymail", "3.0.0", split='validation[:10%]')  # Usar solo el 10% de los datos para validar

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [4]:
# Configurar el modelo y el tokenizer
model_name = "facebook/bart-base"  # Usar una versión más pequeña del modelo
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [5]:
# Función de preprocesamiento y tokenización
def preprocess_function(examples):
    inputs = examples["article"] # obtener articulo
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")  # tokenizar el articulo
    with tokenizer.as_target_tokenizer(): # usar articulo
        labels = tokenizer(examples["highlights"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"] # añadir etiquetas
    return model_inputs

In [6]:
# Tokenizar los conjuntos de datos de entrenamiento y validación; preparamos los datos para el entrenamiento
tokenized_train_dataset = dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = validation_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/28711 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1337 [00:00<?, ? examples/s]

In [7]:
# Parámetros de entrenamiento
training_args = TrainingArguments(
    output_dir="./results", #directorio donde almacenamos resultados
    #evaluation_strategy="no",  # Desactivar evaluación durante el entrenamiento
    learning_rate=3e-5,         # tasa de aprendizaje, sirve para dar pasos mas grandes y acelerar el entrenamiento puede ser malo.
    per_device_train_batch_size=10,  # tamaño del lote de entrenamiento
    per_device_eval_batch_size=10,   # tamaño del lote de evaluación
    num_train_epochs=5,             # numero de epoch
    weight_decay=0.01,              # ayuda con el sobreajuste con modelos que se ajustan demasiado a los datos de entrenamiento.
    gradient_accumulation_steps=4,  # gradiente acumulado, usamos para lotes sin que necesite mas gpu
    fp16=True,                      # Utilizar precisión mixta, nos da aceleracion del entrenamiento, menos uso de memoria.
)

In [8]:
# Entrenador
trainer = Trainer(
    model=model,
    args=training_args, # modelo a entrenar
    train_dataset=tokenized_train_dataset.shuffle(seed=random.randint(0, 1000)),  # Mezclar el conjunto de datos de entrenamiento
    eval_dataset=tokenized_val_dataset, # datos de validacion
)

In [9]:
# Entrenamiento
trainer.train()

Step,Training Loss
500,1.632700
1000,0.957200
1500,0.911800
2000,0.876100
2500,0.851600
3000,0.839700
3500,0.825900


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file

TrainOutput(global_step=3590, training_loss=0.98074923757059, metrics={'train_runtime': 3920.2601, 'train_samples_per_second': 36.619, 'train_steps_per_second': 0.916, 'total_flos': 4.37653586313216e+16, 'train_loss': 0.98074923757059, 'epoch': 5.0})

In [10]:
# Función para resumir texto
def summarize(text):
    # Tokenizar el texto de entrada y mover los tensores al dispositivo del modelo en pytorch porque el modelo BART esta implementado en ese formato
    inputs = tokenizer([text], max_length=512, truncation=True, return_tensors="pt").to(model.device)

    # Generar un resumen con una longitud máxima de 100 y mínima de 30, utilizando un penalty de longitud y beams para búsqueda
    summary_ids = model.generate(
        inputs["input_ids"],  # Pasa los ids
        max_length=100,
        min_length=30,
        length_penalty=2.0, # penalizacion de longitud controla la brevedad del resumen.
        num_beams=2, # tecnica de busqueda que mantiene las mejores secuencias para mejorar la calidad del resumen
        early_stopping=True # detiene la genereracion cuando llega a un token de parada incluso si no se ha llegado al maximo, sin el siempre llegaremos a la maxima longitud 100.
        )

    # Decodificar los IDs del resumen generado y omitir tokens especiales
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [19]:
# Escribimos una sinopsis a ser posible larga
synopsis = """
Don Vito Corleone es el respetado y temido jefe de una de las cinco familias de la mafia de Nueva York en los años 40. El hombre tiene cuatro hijos: Connie, Sonny, Fredo y Michael, que no quiere saber nada de los negocios sucios de su padre. Cuando otro capo, Sollozzo, intenta asesinar a Corleone, empieza una cruenta lucha entre los distintos clanes.
"""

In [23]:
# Resultado
print("Resumen:", summarize(synopsis))

Resumen: Don Vito Corleone es el respetado y temido jefe de las cinco familias de la mafia de Nueva York en los años 40.
Sollozzo empieza una cruenta lucha entre los distintos clanes.
